<a href="https://colab.research.google.com/github/felixzhao/title_catgories_classification/blob/main/JTC_v3_4_Hierarchial_model_implementation_deliverable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install dependency

In [78]:
! pip install sentence-transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# load packages

In [99]:
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict
import json
import pickle

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report

from sentence_transformers import SentenceTransformer, util
from sklearn import metrics

In [ ]:
from typing import Any, Optional

import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import (
    classification_report,
    accuracy_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)
import pickle

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

load data
This work loads data from google drive. Please put the files into "trademe_data" folder under the root of google drive.

Then run the following code to mount the google drive in colab.

You can follow the guide. https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_path = "drive/MyDrive/trademe_data/"

Mounted at /content/drive


# load  data

In [97]:
raw_path = f"{root_path}raw.csv"
c3_dict_path = f"{root_path}c3_dict.json"
c1_dict_path = f"{root_path}c1_dict.json"

c1_mapping_path = f"{root_path}c1_mapping.json"
c2_mapping_path = f"{root_path}c2_mapping.json"
c3_mapping_path = f"{root_path}c3_mapping.json"

In [ ]:
raw_df = pd.read_csv(raw_path)
raw_df.head(1)

,Unnamed: 0,x_title,y_cat_id,cat_1,cat_2,cat_3,x_title_feature
0,0,unqualified asbestos remover,5192,trades & services,labourers,labourers,unqualifi asbesto remov


In [ ]:
raw_df = raw_df[['y_cat_id', 'x_title_feature', 'cat_1', 'cat_2','cat_3']]
print(len(raw_df))
raw_df.head()

20000


,y_cat_id,x_title_feature,cat_1,cat_2,cat_3
0,5192,unqualifi asbesto remov,trades & services,labourers,labourers
1,5123,senior test analyst,it,testing,testing
2,6894,ict trainer supervisor,education,tutoring & training,tutoring & training
3,5197,automot specialist multi facet posit,trades & services,technicians,technicians
4,5114,busi analyst,it,business & systems analysts,business & systems analysts


### load level dicts

In [ ]:
c3_dict = json.load(open(c3_dict_path))
print(len(c3_dict.keys()))
c3_ununique_node_c2_list = c3_dict.keys()
c3_ununique_node_c2_list

3


dict_keys(['accountants', 'drivers & couriers', 'nursing & midwifery'])

In [ ]:
c1_dict = json.load(open(c1_dict_path))
print(len(c1_dict.keys()))
c1_ununique_node_c2_list = c1_dict.keys()
c1_ununique_node_c2_list

10


dict_keys(['administration', 'analysts', 'architects', 'drafting', 'electrical', 'health & safety', 'management', 'project management', 'security', 'supervisors & forepersons'])

## load id and label mapping

In [142]:
c1_mapping = json.load(open(c1_mapping_path))
print(len(c1_mapping.keys()))
c1_mapping

10


{'administration': {'healthcare': 5083, 'office & administration': 5156},
 'analysts': {'banking, finance & insurance': 5022, 'accounting': 6889},
 'architects': {'architecture': 5034, 'it': 5113},
 'drafting': {'architecture': 5035, 'engineering': 5059},
 'electrical': {'engineering': 5060, 'trades & services': 5188},
 'health & safety': {'construction & roading': 4957, 'hr & recruitment': 4958},
 'management': {'automotive': 4962,
  'accounting': 5011,
  'banking, finance & insurance': 5030,
  'customer service': 5047,
  'engineering': 5063,
  'transport & logistics': 5072,
  'healthcare': 5088,
  'hospitality & tourism': 5103,
  'it': 5117,
  'manufacturing & operations': 5136,
  'marketing, media & communications': 5151,
  'retail': 5174},
 'project management': {'engineering': 5065, 'it': 5120},
 'security': {'trades & services': 5196, 'it': 6899},
 'supervisors & forepersons': {'construction & roading': 5042,
  'manufacturing & operations': 6901}}

In [143]:
def read_from_file_pickle(file_name):
    res = None
    with open(file_name, "rb") as file:
        res = pickle.load(file) 
    return res 

In [144]:
c2_mapping = read_from_file_pickle(c2_mapping_path)
print(len(c2_mapping.keys()))

149


In [147]:
c3_mapping = json.load(open(c3_mapping_path))
print(len(c3_mapping.keys()))
c3_mapping

3


{'accountants': {'business advisory services': 621,
  'assistant accountants': 6882,
  'audit': 6883,
  'financial accounting & reporting': 6884,
  'general / ca': 6885,
  'management accountants': 6886,
  'tax': 6887},
 'drivers & couriers': {'couriers': 6908, 'truck ht': 6909},
 'nursing & midwifery': {'community & primary care': 900,
  'critical care & emergency': 904,
  'medical': 906,
  'mental health': 907,
  'paediatrics': 910,
  'surgical': 916,
  'theatre & recovery': 927,
  'women’s health': 929}}

# Model Classes

In [ ]:
class BasicClassifierRFC:
    def __init__(self):
        self.clf = None

    def build(self):
        self.clf = RandomForestClassifier(n_estimators=100)
    
    def train(self, X_train, y_train):
        return self.clf.fit(X_train, y_train)

    def predict(self, input):
        return self.batch_predict([input])

    def batch_predict(self, inputs):
        return self.clf.predict(inputs)

## Root Model

In [ ]:
class RootClassifier:
    def __init__(self, df, classifier=BasicClassifierRFC(), col_X="x_title_feature_encoding", col_y="cat_2", test_size=0.2, random_state=123):
        self.df = df
        self.classifier = classifier
        self.test_size = test_size
        self.random_state = random_state
        self.col_X = col_X
        self.col_y = col_y
        self.accuracy = {}
        self.reports = {}

    def _train_test_split(self):
        X_train, X_test , y_train, y_test = train_test_split(self.df[self.col_X],   self.df[[self.col_y]],  test_size=self.test_size, random_state=self.random_state)
        return X_train.to_list(), X_test.to_list() , y_train, y_test

    def build(self):
        X_train, X_test , y_train, y_test = self._train_test_split()
        self.classifier.build()
        model = self.classifier.train(X_train, y_train)
        # eval
        y_pred = self.classifier.batch_predict(X_test)
        self.accuracy['root'] = metrics.accuracy_score(y_test, y_pred)
        self.reports['root'] = classification_report(y_test, y_pred)
        return model

    def eval(self):
        return self.accuracy, self.reports

## Leaves Model

In [ ]:
class LeavesClassifier:
    def __init__(self, df, root_nodes, col_y, col_X="x_title_feature_encoding", test_size=0.2, random_state=123):
        self.df = df
        self.root_nodes = root_nodes
        self.test_size = test_size
        self.random_state = random_state
        self.col_X = col_X
        self.col_y = col_y
        self.accuracy = {}
        self.reports = {}

    def _train_test_split(self, node):
        node_df = self.df[self.df.cat_2 == node]
        X_train, X_test , y_train, y_test = train_test_split(node_df[self.col_X],   node_df[[self.col_y]],  test_size=self.test_size, random_state=self.random_state)
        return X_train.to_list(), X_test.to_list() , y_train, y_test

    def build(self):
        leaves_models = {}
        for node in self.root_nodes:
            X_train, X_test , y_train, y_test = self._train_test_split(node)
            classifier = BasicClassifierRFC()
            classifier.build()
            leaves_models[node] = classifier.train(X_train, y_train)
            # eval
            y_pred = classifier.batch_predict(X_test)
            self.accuracy[node] = metrics.accuracy_score(y_test, y_pred)
            self.reports[node] = classification_report(y_test, y_pred)
        return leaves_models

    def eval(self):
        return self.accuracy, self.reports

## hierarchical model

In [160]:
class HierarchicalClassifier:
    def __init__(self, root_model, cat_3_models, cat_1_models, c1_mapping, c2_mapping, c3_mapping):
        self.root_model = root_model
        self.cat_3_models = cat_3_models
        self.cat_1_models = cat_1_models
        self.c1_mapping = c1_mapping
        self.c2_mapping = c2_mapping
        self.c3_mapping = c3_mapping
        
    def predict(self, query):
        query = [query]
        root_pred = self.root_model.predict(query)[0]
        print(f"root predict: {root_pred}")
        if root_pred in self.cat_3_models.keys():
            cat_3_predict = self.cat_3_models[root_pred].predict(query)[0]
            print(f"cat 3 pred: {cat_3_predict}")
            return self.c3_mapping[root_pred][cat_3_predict]
        elif root_pred in self.cat_1_models.keys():
            cat_1_predict = self.cat_1_models[root_pred].predict(query)[0]
            print(f"cat 1 pred: {cat_1_predict}")
            return self.c1_mapping[root_pred][cat_1_predict]

        print(f"cat 2 pred: {root_pred}")
        return self.c2_mapping[root_pred]

    def batch_predict(self, queries):
        predictions = []
        for query in queries:
            predictions.append(self.predict(query))
        return predictions

    

# Data Preparation

In [ ]:
data_df = raw_df.copy()

## encoding

In [ ]:
data_df['x_title_feature_encoding'] = data_df['x_title_feature'].apply(lambda x: model.encode(str(x), convert_to_tensor=True).cpu().numpy())

In [ ]:
print(len(data_df))
data_df.head(2)

20000


,y_cat_id,x_title_feature,cat_1,cat_2,cat_3,x_title_feature_encoding
0,5192,unqualifi asbesto remov,trades & services,labourers,labourers,"[-0.04014635, 0.04006017, -0.052789543, 0.0232..."
1,5123,senior test analyst,it,testing,testing,"[-0.02413135, 0.055597737, -0.103181325, 0.024..."


## split for Hierarchial model and eval

In [80]:
train_dataset = data_df.sample(frac = 0.8)
print(len(train_dataset))

16000


In [ ]:
eval_dataset = data_df.drop(train_dataset.index)
print(len(eval_dataset))

4000


# Model training for each level

## C1 models

In [81]:
c1_root_nodes = list(c1_dict.keys())
c1_root_nodes

['administration',
 'analysts',
 'architects',
 'drafting',
 'electrical',
 'health & safety',
 'management',
 'project management',
 'security',
 'supervisors & forepersons']

In [82]:
c1_classifiers = LeavesClassifier(df=train_dataset, root_nodes=c1_root_nodes, col_y="cat_1")
c1_models = c1_classifiers.build()

<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self.clf.fit(X_train, y_train)
<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self.clf.fit(X_train, y_train)
<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self.clf.fit(X_train, y_train)
<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self.clf.fit(X_train, y_train)
<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was 

In [90]:
acc, rep = c1_classifiers.eval()

In [91]:
for k in acc.keys():
    print(k)
    print(acc[k])

administration
0.8357142857142857
analysts
0.6666666666666666
architects
0.95
drafting
0.7777777777777778
electrical
0.8695652173913043
health & safety
0.5625
management
0.5272727272727272
project management
0.9411764705882353
security
0.8636363636363636
supervisors & forepersons
0.85


In [92]:
for k in rep.keys():
    print(k)
    print(rep[k])

administration
                         precision    recall  f1-score   support

             healthcare       1.00      0.18      0.30        28
office & administration       0.83      1.00      0.91       112

               accuracy                           0.84       140
              macro avg       0.91      0.59      0.60       140
           weighted avg       0.86      0.84      0.79       140

analysts
                              precision    recall  f1-score   support

                  accounting       0.75      0.75      0.75         4
banking, finance & insurance       0.50      0.50      0.50         2

                    accuracy                           0.67         6
                   macro avg       0.62      0.62      0.62         6
                weighted avg       0.67      0.67      0.67         6

architects
              precision    recall  f1-score   support

architecture       0.93      1.00      0.97        14
          it       1.00      0.83      0

## C3 models

In [85]:
c3_root_nodes = list(c3_dict.keys())
c3_root_nodes

['accountants', 'drivers & couriers', 'nursing & midwifery']

In [86]:
c3_classifiers = LeavesClassifier(df=train_dataset, root_nodes=c3_root_nodes, col_y="cat_3")
c3_models = c3_classifiers.build()

<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self.clf.fit(X_train, y_train)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and be

In [87]:
acc, rep = c3_classifiers.eval()

In [88]:
for k in acc.keys():
    print(k)
    print(acc[k])

accountants
0.6530612244897959
drivers & couriers
0.9017857142857143
nursing & midwifery
0.5411764705882353


In [89]:
for k in rep.keys():
    print(k)
    print(rep[k])

accountants
                                  precision    recall  f1-score   support

           assistant accountants       0.45      0.62      0.53         8
                           audit       0.71      1.00      0.83         5
      business advisory services       0.60      0.67      0.63         9
financial accounting & reporting       0.71      0.42      0.53        12
                    general / ca       0.00      0.00      0.00         2
          management accountants       0.79      0.92      0.85        12
                             tax       0.00      0.00      0.00         1

                        accuracy                           0.65        49
                       macro avg       0.47      0.52      0.48        49
                    weighted avg       0.62      0.65      0.62        49

drivers & couriers
              precision    recall  f1-score   support

    couriers       1.00      0.48      0.65        21
    truck ht       0.89      1.00      0.94

## Root model

In [93]:
root_classifier = RootClassifier(df=train_dataset)
root_model = root_classifier.build()

<ipython-input-12-361938b15d1b>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self.clf.fit(X_train, y_train)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and be

In [94]:
acc, rep = root_classifier.eval()

In [95]:
print(f"accuracy of root: {acc['root']}")

accuracy of root: 0.5825


In [96]:
print(rep['root'])

                                             precision    recall  f1-score   support

                         account management       0.67      0.33      0.44         6
                                accountants       0.65      0.77      0.70        43
                    accounts administrators       0.89      0.47      0.62        17
                           accounts payable       0.50      0.50      0.50         6
                        accounts receivable       0.00      0.00      0.00         1
                             administration       0.36      0.75      0.49       130
                                advertising       1.00      0.50      0.67         2
                   air con. & refrigeration       0.88      0.64      0.74        11
                                   analysts       0.50      0.17      0.25         6
                                 architects       0.74      0.82      0.78        17
                              area managers       0.00      0.00

# Hierarcial model

In [ ]:
X_eval = eval_dataset["x_title_feature_encoding"].to_list()
y_eval = eval_dataset["y_cat_id"]

In [161]:
hierarcial_classifier = HierarchicalClassifier(
    root_model= root_model, cat_3_models= c3_models, cat_1_models = c1_models,
    c1_mapping = c1_mapping, c2_mapping = c2_mapping, c3_mapping = c3_mapping
    )

## eval of singal predict

In [150]:
hierarcial_classifier.c2_mapping['bar staff & baristas']

5098

In [164]:
from datetime import datetime
start = datetime.utcnow()

preds = hierarcial_classifier.predict(X_eval[0])

date= datetime.utcnow() - start
seconds =(date.total_seconds())
milliseconds = round(seconds*1000)
print("Milliseconds since epoch:",milliseconds)

root predict: bar staff & baristas
cat 2 pred: bar staff & baristas
Milliseconds since epoch: 19


In [152]:
preds

5098

In [153]:
y_eval.head(1)

12    5098
Name: y_cat_id, dtype: int64

## batch eval

In [154]:
hierarcial_classifier.c1_mapping.keys()

dict_keys(['administration', 'analysts', 'architects', 'drafting', 'electrical', 'health & safety', 'management', 'project management', 'security', 'supervisors & forepersons'])

In [163]:
# for x in X_eval[:1000]:
#     hierarcial_classifier.predict(x)

In [162]:
y_pred = hierarcial_classifier.batch_predict(X_eval)

Streaming output truncated to the last 5000 lines.
root predict: telesales
cat 2 pred: telesales
root predict: process & assembly
cat 2 pred: process & assembly
root predict: labourers
cat 2 pred: labourers
root predict: dentistry
cat 2 pred: dentistry
root predict: administration
cat 1 pred: office & administration
root predict: kitchen staff
cat 2 pred: kitchen staff
root predict: caregiving
cat 2 pred: caregiving
root predict: building & carpentry
cat 2 pred: building & carpentry
root predict: administration
cat 1 pred: office & administration
root predict: community & social services
cat 2 pred: community & social services
root predict: gardening & landscaping
cat 2 pred: gardening & landscaping
root predict: chefs
cat 2 pred: chefs
root predict: early childhood
cat 2 pred: early childhood
root predict: department managers
cat 2 pred: department managers
root predict: management
cat 1 pred: hospitality & tourism
root predict: early childhood
cat 2 pred: early childhood
root predict

In [165]:
metrics.accuracy_score(y_eval, y_pred)

0.7925

In [167]:
print(classification_report(y_eval, y_pred))

              precision    recall  f1-score   support

         621       0.64      0.44      0.52        16
         674       1.00      1.00      1.00         2
         728       1.00      0.89      0.94        18
         733       0.87      0.52      0.65        25
         748       1.00      0.83      0.91         6
         749       1.00      0.80      0.89         5
         751       1.00      0.55      0.71        11
         752       1.00      0.83      0.91         6
         755       1.00      0.80      0.89         5
         759       0.67      0.67      0.67         6
         761       0.89      0.78      0.83        32
         763       1.00      1.00      1.00         2
         764       0.80      0.57      0.67         7
         765       1.00      0.50      0.67         2
         768       1.00      0.79      0.88        14
         883       1.00      0.67      0.80         9
         897       0.95      0.95      0.95        20
         900       0.60    

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
